In [1]:
%%capture
! pip install -q --upgrade pypdf llama-index llama-index-llms-gemini llama-index-embeddings-gemini

In [2]:
## get the data
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-06-29 15:13:16--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  56.1MB/s    in 0.3s    

2024-06-29 15:13:16 (56.1 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



In [3]:
import os
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import StorageContext
from llama_index.core import SimpleDirectoryReader
import nest_asyncio

nest_asyncio.apply()

In [5]:
##Load the data
documents = SimpleDirectoryReader(input_files=['metagpt.pdf']).load_data()

In [7]:
gemini_embedding_model = GeminiEmbedding(model_name="models/embedding-001")
llm = Gemini(model_name="models/gemini-1.5-flash-latest")

In [8]:
# Set Global settings
Settings.llm = llm
Settings.embed_model = gemini_embedding_model

In [11]:
index = VectorStoreIndex(documents)

In [12]:
query = "How do agents share information with other agents?"

base_query_engine = index.as_query_engine()
response = base_query_engine.query(query)
print(response)

Agents share and comply with development and communication protocols defined in MetaGPT. 



## Retry Query Engine

In [13]:
from llama_index.core.query_engine import RetryQueryEngine
from llama_index.core.evaluation import RelevancyEvaluator

query_response_evaluator = RelevancyEvaluator()
retry_query_engine = RetryQueryEngine(
    base_query_engine, query_response_evaluator
)
retry_response = retry_query_engine.query(query)
print(retry_response)

Agents share and comply with development and communication protocols defined in MetaGPT. 



In [14]:
from llama_index.core.query_engine import RetrySourceQueryEngine

retry_source_query_engine = RetrySourceQueryEngine(
    base_query_engine, query_response_evaluator
)
retry_source_response = retry_source_query_engine.query(query)
print(retry_source_response)

Agents share and comply with development and communication protocols defined in MetaGPT. 



In [15]:
from llama_index.core.evaluation import GuidelineEvaluator
from llama_index.core.evaluation.guideline import DEFAULT_GUIDELINES
from llama_index.core import Response
from llama_index.core.indices.query.query_transform.feedback_transform import (
    FeedbackQueryTransformation,
)
from llama_index.core.query_engine import RetryGuidelineQueryEngine

# Guideline eval
guideline_eval = GuidelineEvaluator(
    guidelines=DEFAULT_GUIDELINES
    + "\nThe response should not be overly long.\n"
    "The response should try to summarize where possible.\n"
)  # just for example

In [16]:
##under the hood
typed_response = (
    response if isinstance(response, Response) else response.get_response()
)
eval = guideline_eval.evaluate_response(query, typed_response)
print(f"Guideline eval evaluation result: {eval.feedback}")

feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
transformed_query = feedback_query_transform.run(query, {"evaluation": eval})
print(f"Transformed query: {transformed_query.query_str}")

Guideline eval evaluation result: The response is vague and does not fully answer the query. It mentions MetaGPT but does not explain how agents share information within it. The response lacks specifics and does not provide any statistics or numbers. It is also overly short and does not summarize the information.
Transformed query: Here is a previous bad answer.
Agents share and comply with development and communication protocols defined in MetaGPT. 

Here is some feedback from the evaluator about the response given.
The response is vague and does not fully answer the query. It mentions MetaGPT but does not explain how agents share information within it. The response lacks specifics and does not provide any statistics or numbers. It is also overly short and does not summarize the information.
Now answer the question.
Please provide more context about the agents you are referring to.  For example:

* **What kind of agents are these?** Are they software agents, AI agents, human agents, o

In [17]:
retry_guideline_query_engine = RetryGuidelineQueryEngine(
    base_query_engine, guideline_eval, resynthesize_query=True
)
retry_guideline_response = retry_guideline_query_engine.query(query)
print(retry_guideline_response)

Agents in MetaGPT communicate through a shared message pool, where they publish structured messages that other agents can subscribe to based on their profiles. This allows agents to obtain directional information from other roles and public information from the environment. For example, an Architect might publish a message containing the system design components, such as File Lists, Data Structures, and Interface Definitions. Engineers, who are subscribed to messages related to system design, can then access this information to execute the designated classes and functions. Similarly, a Product Manager might publish a message containing user requirements, which the Architect can then use to translate into system design components. This structured communication protocol ensures efficient information sharing and collaboration among agents, enabling them to work together effectively to complete complex tasks. 

